# Get the Top 5 Chunks to Check Paper's Stance

## Step 0: Environment Setup

In [7]:
from dotenv import load_dotenv # type: ignore
import os
from langchain_neo4j import Neo4jGraph # type: ignore
from libs import create_vector_index
import pandas as pd # type: ignore
from conn import connect2Googlesheet
from annolibs import get_all_chunks_per_paper, compare_embeddings
from openai import OpenAI
load_dotenv()

True

In [8]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

# Check if the entities index exists
index_name = "entities"
query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR' AND name = $index_name"

result = graph.query(query, params={"index_name": index_name})
if result:
    print("The 'entities' index already exists.")
else:
    create_vector_index(graph, "entities")

Connected to Neo4j database successfully.
The 'entities' index already exists.


## Step 1: Load Questions from Google Sheet

In [9]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(2)  

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_Paper = pd.DataFrame(data)
df_Paper.head()

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blocking agents increases the ventilator free days?,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?","Maybe this question: In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?",fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have a significance difference in mortality rate beween patients who recieved an early and continous cisatracurium infusion than those with usual care approach with lighter sedation targets?,Local question (not sure if this is the aim of your project) It will be nice as second step after proving the general summarization is working but focusing in general summarization would be priority in my opinion so you can have meanigful tool.,fixed,Wrong concept since PEEP by itself is mandatory component in ventilator.,"Does the use of neuromuscular blockers in patients with moderate-to-severe ARDS impact cardiovascular stability, particularly in terms of vasopressor requirements and hemodynamic effects, compared to sedation strategy without routine neuromuscular blockade?"
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservative fluid management strategy improves lung function and decrease ventilator days compared to liberal strategy?","Local question (not sure if this is the aim of your project) consider (WikiJournal): In patients with ALI/ARDS that are intubated and receiving positive pressure ventilation, how does the conservative compare to the liberal fluid management strategy in reducing mortality?",fixed,Check if studies defined conservative by CVP < 4 or elese just dont mention how much the CVP (i prefer the last approach),


## Step 3: Get the Chunks from Each Paper

In [10]:
# Filter papers with 'ARDS' in condition
ARDS_papers = df_Paper[df_Paper['condition'].str.contains('ARDS', case=False, na=False)]

# Display the filtered papers
print(f"Found {len(ARDS_papers)} papers related to ARDS:")


# Get unique paper names
ARDS_paper_names = ARDS_papers['docs'].str.strip().unique()
print(f"Number of unique paper names: {len(ARDS_paper_names)}")
ARDS_papers_chunks = get_all_chunks_per_paper(graph, ARDS_paper_names)

Found 37 papers related to ARDS:
Number of unique paper names: 33
Found 100 chunks in paper ACURASYS
Found 99 chunks in paper ROSE
Found 100 chunks in paper FACTT
Found 100 chunks in paper ARDSNet
Found 100 chunks in paper PROSEVA
Found 100 chunks in paper OSCILLATE
Found 100 chunks in paper ESCPARDS
Found 100 chunks in paper LCOTARDS
Found 100 chunks in paper ETSDMV
Found 98 chunks in paper SMDLTSWCI
Found 100 chunks in paper PESDCOMV
Found 100 chunks in paper LSPA
Found 100 chunks in paper NBSARDS
Found 100 chunks in paper TOF-ARDS
Found 100 chunks in paper EDPMARDSLPMV
Found 100 chunks in paper ARDSSRDRFMS
Found 100 chunks in paper FMWSCPARDS
Found 0 chunks in paper PAC-in-FACTT
Found 52 chunks in paper DIHRFARDSNET
Found 100 chunks in paper DLASPEEPvsARDSNET
Found 100 chunks in paper BMIMSARDS
Found 100 chunks in paper PPATDSHMA
Found 100 chunks in paper CEIIUPPSARDS
Found 100 chunks in paper PVOEMVRARDS
Found 100 chunks in paper CHEST
Found 100 chunks in paper ENB
Found 100 chunks

In [11]:
# check if the expected number of papers match the actual number of papers
# Get list of expected papers
expected_papers = ARDS_paper_names.tolist()

# Get list of actual papers from chunks_of_paper directory
actual_papers = [f.replace('chunks_of_', '').replace('.csv', '') 
                 for f in os.listdir('./chunks_of_paper') 
                 if f.endswith('.csv')]

# Find missing papers
missing_papers = set(expected_papers) - set(actual_papers)

print("Missing papers:")
for paper in missing_papers:
    print(f"- {paper}")

Missing papers:
- CCCARDS
- PAC-in-FACTT


## Step 4: Compare Question Embedding and Paper Chunk Embeddings

In [23]:
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

client = OpenAI(api_key=api_key)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

idx = 0 # index of the question. Change this to test different questions
test_question = df_Paper['Question'].iloc[idx]
print(f"Question {idx+1}: {test_question}")

for i, paper in enumerate(ARDS_paper_names):
    print(f"\nPaper {i+1}: {paper}")
    paper_name = str(paper)  # without .pdf extension
    top5chunks = compare_embeddings(
        question=test_question,
        paper=paper_name,
        top_k=5
    )
    
    prompt = f"Does the following question have any relevance to the text chunks below? Keep your answer short\n\nQuestion: {test_question}\n\nText Chunks:\n" + \
         '\n'.join([f"{i+1}. {text}" for i, text in enumerate(top5chunks['chunk_text'])])
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200
        )
        output = response.choices[0].message.content.strip()
        display(output)
        display("The Avg similarity of the top5chunks is:", top5chunks["similarity_score"].mean())
        display(top5chunks)
    except Exception as e:
        print(f"Error: {e}")
        continue

    # output = response.choices[0].message.content.strip()
    # display(output)

Question 1: Does early administration of neuromuscular blocking agents increases the ventilator free days?

Paper 1: ACURASYS


'Yes, the question is relevant as the text chunks discuss the effects of early administration of neuromuscular blocking agents on patients with ARDS, including the impact on ventilator use.'

'The Avg similarity of the top5chunks is:'

0.6337400000000001

,paper_name,position,chunk_text,similarity_score
63,ACURASYS.pdf,64,"Neuromuscular Blocking Agents in ARDS n engl j med 363;12 nejm.org september 16, 2010 1111 without the aid of a ventilator, for a period of at least 48 hours for weaning from the ventilator to be considered successful. The number of ventilator- free days was considered to be zero for patients who were weaned from mechanical ventilation but who died before day 28 or",0.7262
34,ACURASYS.pdf,35,"Neuromuscular Blocking Agents in ARDS n engl j med 363;12 nejm.org september 16, 2010 1109 sure were adjusted as in the Prospective, Random- ized, Multi-Center Trial of 12 ml/kg Tidal Volume Positive Pressure Ventilation for Treatment of Acute Lung Injury and Acute Respiratory Distress Syndrome (ARMA).14 An open-label, rapid, intravenous injection",0.6263
10,ACURASYS.pdf,11,"mechanical ventilation for the acute respiratory distress syn- drome (ARDS), neuromuscular blocking agents may improve oxygenation and de- crease ventilator-induced lung injury but may also cause muscle weakness. We evalu- ated clinical outcomes after 2 days of therapy with neuromuscular blocking agents in patients with early, severe ARDS. Methods In this multicenter, double-blind trial, 340 patients presenting to the intensive care unit (ICU) with an onset",0.6136
22,ACURASYS.pdf,23,"did not receive a blocking agent; P = 0.06). However, this study was not designed or powered to evaluate mortality. Thus, the benefits and risks of adjunctive therapy with neuromuscu- lar blocking agents in patients with ARDS who were receiving lung-protective mechanical venti- lation14 require further evaluation. We conducted a multicenter, randomized, pla- cebo-controlled, double-blind trial to determine",0.6120
16,ACURASYS.pdf,17,"�= 0.05). The rate of ICU-acquired paresis did not differ significantly between the two groups. Conclusions In patients with severe ARDS, early administration of a neuromuscular blocking agent improved the adjusted 90-day survival and increased the time off the ventilator without increasing muscle weakness. (Funded by Assistance Publique–Hôpitaux de Marseille and the Programme Hospitalier de Recherche Clinique Rég",0.5906


...


Paper 33: PTARDS


'No, the question is not relevant to the text chunks provided.'

'The Avg similarity of the top5chunks is:'

0.39396

,paper_name,position,chunk_text,similarity_score
16,PTARDS.pdf,17,"Copyright © 2016 by the Shock Society. Unauthorized reproduction of this article is prohibited. studies to identify mechanisms of action. Secondly, we con- ducted a meta-analysis on the available clinical studies to determine the effect of ASA on important clinical outcomes in ARDS (e.g., prevention, ventilator-free days, survival). MATERIALS AND METHODS Study selection, data extraction, and quality assessment A systemic search strategy was conducted to capture published preclinical and clinical studies",0.4339
64,PTARDS.pdf,65,"resistance, and early pulmonary gas exchange. However, the late increase in pulmonary artery pressure and airway resistance was not inhibited by ASA and the late severe deterioration of gas exchange was even exaggerated compared with controls. In the study of Goncalves de Moraes et al. (29), it was shown that pretreatment with ASA increased the neutrophil number in the bronchial alveolar fluid (BALF). Quality assessment of preclinical studies Two studies (32, 35) reported",0.3931
86,PTARDS.pdf,87,"level III evidence. The only RCTon the effect of APT for the prevention of ARDS was performed by Vincent et al. (52) in 1985. In this small (n ¼ 33) double-blinded pilot RCT, patients with circu- latory shock received either ASA plus dipyridamole or ASA plus placebo, the active comparator thus being the addition of dipyridamole, and was therefore excluded from our analysis. Of note, no benefits",0.3858
30,PTARDS.pdf,31,"a combination of pretreatment and treatment for lung injury. All models induced lung injury, evidenced by an increase in lung edema and the appearance of neutrophils, proteins, and inflammatory mediators in broncho-alveolar lavage fluid. In two studies respiratory mechanics (27, 28) and in three studies pulmonary and systemic hemodynamics were measured (26– 28). Of the 15 preclinical trials, 13 reported a beneficial effect of antiplatelet drug on ARDS (26, 27",0.3800
31,PTARDS.pdf,32,"clinical trials, 13 reported a beneficial effect of antiplatelet drug on ARDS (26, 27, 30–40), evidenced by improved oxygenation, diminished lung edema, inflammation, and in some an increased survival (32, 36). No benefit was observed in one study (28) and in one study (29) worsening of inflammation was demonstrated. Beneficial mechanisms of action of ASA in preclinical studies Leeman et al. (26) reported a beneficial effect of ASA. The involved",0.3770
